In [22]:
import numpy as np
import pandas as pd
import networkx as nx
import os

In [25]:
input_file = 'Weighted_paths_0.65_corrected' # this is the 'output_file_1.evaluated.filtered_0.67' file
ranking_file = './ranks/alpha_0.65_corrected.desc'
N = 100 # top ranked genes in the network
output_file = 'subnetwork_0.65_corrected_'+str(N)+'.js'
output_file_components_directory = './components/subnetwork_0.65_corrected_'+str(N)+'/'

In [3]:
df = pd.read_csv(input_file,sep='[',skiprows=0,header=None)

In [4]:
df = df.rename(columns={0: 'path_weight', 1: 'genes_path'})

In [5]:
df

,path_weight,genes_path
0,2.3128625134246903e-9-,"'TRIM69','RPS27A','IRAK1']."
1,2.3142540708481324e-9-,"'EPN1','RPS27A','IRAK1']."
2,2.316282487956147e-9-,"'TRIM69','RPS27A','RFC5']."
3,2.3177228295676374e-9-,"'EPN1','RPS27A','RFC5']."
4,2.3475185172454296e-9-,"'LYN','RPS27A','IRAK1']."
...,...,...
11016825,0.0002110144107775081-,"'PIGG','PIGF','PIGO']."
11016826,0.00021828251598526128-,"'CLCNKA','BSND','CLCNKB']."
11016827,0.00024311395395978854-,"'APOB','STAB1','SPARC']."
11016828,0.0002728755664282789-,"'EP300','LCN2','SLC22A17']."


In [6]:
df[['gene1','gene2','gene3']] = df['genes_path'].str.split(',',expand=True)
df[['gene3','brol']] = df['gene3'].str.split(']',expand=True)
df = df.drop(['brol','genes_path'], axis=1)
df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)
df = df.drop(['brol'], axis=1)
df

/tmp/ipykernel_11868/271992433.py:4: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)


,path_weight,gene1,gene2,gene3
0,2.3128625134246903e-9,'TRIM69','RPS27A','IRAK1'
1,2.3142540708481324e-9,'EPN1','RPS27A','IRAK1'
2,2.316282487956147e-9,'TRIM69','RPS27A','RFC5'
3,2.3177228295676374e-9,'EPN1','RPS27A','RFC5'
4,2.3475185172454296e-9,'LYN','RPS27A','IRAK1'
...,...,...,...,...
11016825,0.0002110144107775081,'PIGG','PIGF','PIGO'
11016826,0.00021828251598526128,'CLCNKA','BSND','CLCNKB'
11016827,0.00024311395395978854,'APOB','STAB1','SPARC'
11016828,0.0002728755664282789,'EP300','LCN2','SLC22A17'


In [7]:
df_ranking = pd.read_csv(ranking_file,sep='[',skiprows=0,header=None)

In [8]:
df_ranking_top = df_ranking[0:N]
df_ranking_top
top_genes = df_ranking_top[0].values.tolist()
top_genes2 = []
for i in top_genes:
    a = '\''+i+'\''
    top_genes2.append(a)

In [9]:
df_final = df[(df['gene1'].isin(top_genes2) & df['gene2'].isin(top_genes2)) | (df['gene1'].isin(top_genes2) & df['gene3'].isin(top_genes2)) | (df['gene2'].isin(top_genes2) & df['gene3'].isin(top_genes2))]
df_final

,path_weight,gene1,gene2,gene3
107,2.5571949965174566e-9,'REST','EP300','PCNA'
161,2.6037174628783603e-9,'REST','EP300','MEF2C'
408,2.682672687820205e-9,'CDC42','EP300','AJUBA'
415,2.682672687820205e-9,'CEBPB','EP300','AJUBA'
428,2.682672687820205e-9,'CREB1','EP300','AJUBA'
...,...,...,...,...
11012206,7.182665581434695e-6,'GNAI2','C3','CFHR5'
11012272,7.255854197532197e-6,'GNAI1','C3','CFHR1'
11012480,7.525265984815691e-6,'ITGB2','C3','CFHR1'
11012484,7.535693710250427e-6,'ITGAM','C3','CFHR1'


In [10]:
edges = {}
Normalization = sum(df['path_weight'].astype(float)) # Explained in the paper of OMEN
for i in range(N):
    print(i)
    for j in range(N-1-i):
        node1 = top_genes[i]
        node2 = top_genes[i+1+j]
        nodes = []
        nodes.append(top_genes2[i])
        nodes.append(top_genes2[i+1+j])
        df_edge = df_final[ (df_final['gene1'].isin(nodes) & df_final['gene2'].isin(nodes)) | (df_final['gene1'].isin(nodes) & df_final['gene3'].isin(nodes)) | (df_final['gene2'].isin(nodes) & df_final['gene3'].isin(nodes))]
        s = sum(df_edge['path_weight'].astype(float))/Normalization
        if s>0:
            edges[(node1,node2)] = s*100000 # to have bigger values

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [11]:
print(len(edges))
edges

1968


{('EP300', 'STAT1'): 5.498173969387239,
 ('EP300', 'STAT2'): 6.4868399478466285,
 ('EP300', 'PRKACA'): 3.0156704239278382,
 ('EP300', 'TP53'): 3.695790640510774,
 ('EP300', 'CTCF'): 4.084368075266448,
 ('EP300', 'MYC'): 3.303647943238049,
 ('EP300', 'CTNNB1'): 2.852898971821856,
 ('EP300', 'SP1'): 3.134445488898977,
 ('EP300', 'RPS27A'): 2.3218248280137024,
 ('EP300', 'PRKACB'): 2.711365356619162,
 ('EP300', 'PRKACG'): 2.722197044558396,
 ('EP300', 'STAT3'): 2.8195629173723082,
 ('EP300', 'EGR1'): 3.451131859301416,
 ('EP300', 'E2F1'): 4.260597478209735,
 ('EP300', 'GPLD1'): 0.05471264991446312,
 ('EP300', 'TCF12'): 4.121129360384362,
 ('EP300', 'JUN'): 2.6375853109812124,
 ('EP300', 'FOS'): 2.9792073976629116,
 ('EP300', 'ESR1'): 2.989461047206316,
 ('EP300', 'REST'): 4.173640593332943,
 ('EP300', 'FOXA1'): 3.5781272915050324,
 ('EP300', 'RAD21'): 2.831079922738944,
 ('EP300', 'NFKB1'): 2.3602678594949165,
 ('EP300', 'JUND'): 3.0350041519087756,
 ('EP300', 'YY1'): 3.073491023477529,
 

In [12]:
with open(output_file,'w') as f:
    f.write('graph = {')
    f.write('\n')
    f.write('    nodes: [')
    f.write('\n')
    for i in range(N):
        f.write('        {') 
        f.write('\n')
        f.write('            id: ' + top_genes2[i] + ',') 
        f.write('\n')
        f.write('            rank: ' + str(1+i) + ',') 
        f.write('\n')
        f.write('        },') 
        f.write('\n')
    f.write('    ],')
    f.write('\n')
    f.write('    links: [')
    f.write('\n')
    for edge in edges:
        f.write('        {source: "'+ edge[0] +'", target: "'+edge[1]+'", type: "pp", direction: "directed", max_cost: '+ str(edges[edge]) +', evidence: ""},')
        f.write('\n')
    f.write('    ],')
    f.write('\n')
    f.write('}')
f.close()

In [13]:
def componentsFilteredEdges(edges,Filter):
    array1 = []
    for edge in edges:
        if edges[edge]>Filter:
            array1.append(edge)
    G = nx.Graph()
    G.add_edges_from(array1)
    r = list(nx.connected_components(G))
    return r

In [26]:
def writeComponents(edges,Filters):
    for f in Filters:
        components = componentsFilteredEdges(edges,f)
        file = 'components_' + str(f)
        with open(output_file_components_directory+file,'w') as f:
            i = 1
            for c in components:
                f.write('Component'+str(i)+': ')
                f.write(str(c))
                f.write('\n')
                i = i + 1

In [32]:
path = os.path.join(output_file_components_directory)
os.mkdir(path)
writeComponents(edges,[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7])